In [ ]:
# |default_exp training

In [ ]:
# |export
import json
import logging
from collections.abc import Iterator
from functools import partial
from itertools import cycle
from pathlib import Path
from threading import Thread
from time import sleep, time

import torch
import wandb
from fastcore.basics import store_attr
from fastcore.foundation import L, Self
from human_id import generate_id
from torch.multiprocessing import Manager, Queue

from tmenv.gbx import map_from_uid
from tmlearn.callback import Callback, Callbackable
from tmlearn.collector import (
    ActionPublisherCallback,
    CollectorWorker,
    SessionPublisherCallback,
    StepPublisherCallback,
    WandbCollectorCallback,
    EigenCamPublisherCallback,
)
from tmlearn.net import WebsocketServer
from tmlearn.utils import Timer, format_date, format_duration

# Training
> Manager coordinating a training

In [ ]:
# |export
class Training(Callbackable):
    def __init__(
        self,
        map_scheduler,
        envs,
        agent,
        replay_buffer,
        session_scheduler,
        collector_cbs=[],
        training_cbs=[],
        batch_size=32,
        burnin=1024,
        online=True,  # Activate websocket and wandb logging
        log_every=2,
        ws_port=2013,
        wandb_project="tmaifoo",
        wandb_entity="trackmania-ai",
        wandb_load_group=None,
        wandb_load_agent="agent-latest.pth",
    ):
        super().__init__(training_cbs)
        store_attr(but="training_cbs")

        if self.online:
            self.wandb_api = wandb.Api()
            self.ws_server = WebsocketServer(port=self.ws_port)
            self.init_wandb_run()
            self.add_cb(WandbSaveAgentSlidingCallback)
            self.add_cb(WandbSaveAgentPersistentCallback)
            self.load_state_dict(json.loads(self.wandb_run.summary.get("state_dict")))
            self.log_timer = Timer(self.log_every)

        self.map_scheduler.start()
        self.collectors_worker = L(envs).map(self.build_worker)
        self.collectors_worker.map(Self.start())

    def fit(self):
        self("before_fit")
        while len(self.replay_buffer) < self.burnin:
            sleep(0.1)
            self.replay_buffer.empty_queue()
            if self.online:
                self.wandb_run.summary["state_dict"] = json.dumps(self.state_dict())
        logging.info("Burnin completed. Time to learn !")
        self("after_burnin")
        while True:
            self.replay_buffer.empty_queue()
            self("before_fit_one_batch")
            batch, indices, weights = self.replay_buffer.sample(self.batch_size)
            errors, metrics = self.agent.fit_one_batch(batch, weights)
            self("after_fit_one_batch")
            self.replay_buffer.update_priorities(indices, errors)
            if self.online and self.log_timer():
                self.wandb_run.log(metrics)
                self.wandb_run.summary["state_dict"] = json.dumps(self.state_dict())
                self.log_timer.reinit()

    def init_wandb_run(self):
        if self.wandb_load_group is not None:
            training_runs = self.training_runs(self.wandb_load_group)
            assert (
                len(training_runs) == 1
            ), f"Found {len(training_runs)} training run(s) found, must be exactly one"
            self.wandb_run = wandb.init(
                project=self.wandb_project,
                entity=self.wandb_entity,
                group=self.wandb_load_group,
                id=training_runs[0].id,
                resume="must",
            )
        else:
            while self.training_runs(group := generate_id(word_count=3)):
                pass
            self.wandb_run = wandb.init(
                project=self.wandb_project,
                entity=self.wandb_entity,
                group=group,
                name=f"Training {group}",
                job_type="training",
                tags=["training"] + self.agent.tags,
            )
            self.wandb_run.summary["state_dict"] = json.dumps(self.state_dict())
        self.save_dir = Path(self.wandb_run.dir) / "agent"
        logging.debug(f"Saving agents locally to {self.save_dir}")
        self.save_dir.mkdir(parents=True, exist_ok=True)
        if self.wandb_load_group is not None and self.wandb_load_agent is not None:
            self.load_agent(self.wandb_load_agent)
            self.save_agent("last-loaded", overwrite_latest=False)

    def save_agent(self, raw_name, state_dict=None, overwrite_latest=True):
        if state_dict is None:
            state_dict = self.agent.state_dict()
        torch.save(
            state_dict,
            self.save_dir / f"agent-{raw_name}.pth",
        )
        if overwrite_latest:
            torch.save(
                state_dict,
                self.save_dir / f"agent-latest.pth",
            )
        self.wandb_run.save("agent/*.pth", policy="now")

    def load_agent(self, file, run_path=None):
        if run_path is None:
            run_path = self.wandb_run.path
        state_dict = torch.load(wandb.restore(f"agent/{file}", run_path=run_path).name)
        self.agent.load_state_dict(state_dict)

    @property
    def wandb_path(self):
        return f"{self.wandb_entity}/{self.wandb_project}"

    def training_runs(self, group):
        filters = dict(group=group, tags="training")
        return L(self.wandb_api.runs(self.wandb_path, filters))

    def build_worker(self, env):
        online_cbs = []
        if self.online:
            online_cbs += [
                partial(
                    WandbCollectorCallback,
                    project=self.wandb_project,
                    entity=self.wandb_entity,
                    group=self.wandb_run.group,
                ),
                partial(
                    ActionPublisherCallback,
                    self.ws_server.add_publisher(f"{env.name}-action"),
                ),
                partial(
                    StepPublisherCallback,
                    self.ws_server.add_publisher(f"{env.name}-step"),
                ),
                partial(
                    SessionPublisherCallback,
                    self.ws_server.add_publisher(f"{env.name}-session"),
                ),
                # partial(
                #     EigenCamPublisherCallback,
                #     self.ws_server.add_publisher(f"{env.name}-cam"),
                # ),
            ]

        return CollectorWorker(
            env,
            self.collector_cbs + online_cbs,
            self.agent,
            self.replay_buffer,
            self.session_scheduler,
            self.map_scheduler.next_queue,
            self.map_scheduler.done_queue,
        )

    def state_dict(self):
        return dict(
            map_scheduler=self.map_scheduler.state_dict(), cbs=self.cbs_state_dict()
        )

    def load_state_dict(self, state_dict):
        self.map_scheduler.load_state_dict(state_dict["map_scheduler"])
        if "cbs" in state_dict:
            self.load_cbs_state_dict(state_dict["cbs"])

In [ ]:
# |export
class MapScheduler:
    def __init__(self, maps_uid):
        assert 0 < len(maps_uid)
        self.maps_uid = maps_uid
        self.next_maps = L()  # maps to be queued
        self.queued_maps = L()  # queued maps not done yet
        self.next_queue = Queue(1)
        self.done_queue = Queue()

    def start(self):
        Thread(target=self.next_maps_target).start()
        Thread(target=self.done_maps_target).start()

    def next_maps_target(self):
        while True:
            if not self.next_maps:
                self.next_maps.extend(self.maps_uid)
            map_uid = self.next_maps.pop(0)
            if map_uid in self.queued_maps:
                logging.warning(
                    f"Skipping map uid {map_uid}, map is already being played"
                )
                continue
            self.queued_maps.append(map_uid)
            self.next_queue.put(map_uid)

    def done_maps_target(self):
        while True:
            self.queued_maps.remove(self.done_queue.get())

    def state_dict(self):
        return dict(next_maps=list(self.next_maps), queued_maps=list(self.queued_maps))

    def load_state_dict(self, state_dict):
        self.next_maps = L(state_dict["queued_maps"] + state_dict["next_maps"])

class OneMapScheduler:
    def __init__(self, map_uid):
        self.map_uid = map_uid
        self.next_queue = Queue(100)
        while not self.next_queue.full():
            self.next_queue.put(map_uid)
        self.done_queue = Queue()

    def start(self):
        pass

    def state_dict(self):
        pass

    def load_state_dict(self, state_dict):
        pass

In [ ]:
# |export
class ExperienceTimeoutException(Exception):
    pass


class ExperienceTimeoutCallback(Callback):
    def __init__(self, timeout=5 * 60):
        store_attr()

    def before_fit_one_batch(self):
        if self.timeout < time() - self.parent.replay_buffer.last_insert_time:
            self.parent.collectors_worker.map(Self.kill())
            raise ExperienceTimeoutException(
                f"Replay buffer received no new experiences for more than {format_duration(self.timeout)}."
            )

In [ ]:
# |export
class LrSchedulerCallback(Callback):
    def __init__(self, scheduler, max_time):
        store_attr()
        self.offset = 0
        self.start_time = time()

    def before_fit(self):
        self.start_time = time() - self.offset

    def before_fit_one_batch(self):
        lr = self.scheduler(min(1, (time() - self.start_time) / self.max_time))
        self.parent.agent.set_lr(lr)

    def state_dict(self):
        return dict(offset=time() - self.start_time)

    def load_state_dict(self, state_dict):
        self.offset = state_dict["offset"]

In [ ]:
# |export
class WandbSaveAgentSlidingCallback(Callback):
    def __init__(self, every=3600, nb_save=25):
        store_attr()
        self.timer = Timer(every)
        self.agents = []

    def before_fit_one_batch(self):
        if self.timer():
            self._save()
            self.timer.reinit()

    def _save(self):
        self.agents.insert(0, self.parent.agent.state_dict())
        self.agents = self.agents[: self.nb_save]
        for index, state_dict in enumerate(self.agents):
            self.parent.save_agent(
                format_duration(index * self.every), state_dict, index == 0
            )


class WandbSaveAgentPersistentCallback(Callback):
    def __init__(self, every=48 * 3600):
        store_attr()
        self.offset = 0
        self.start_time = time()

    def before_fit(self):
        self.start_time = time() - self.offset
        self.timer = Timer(self.every - (self.offset % self.every))

    def before_fit_one_batch(self):
        if self.timer():
            self.parent.save_agent(format_date("%Y-%m-%d_%H-%M"))
            self.timer = Timer(self.every)

    def state_dict(self):
        return dict(offset=time() - self.start_time)

    def load_state_dict(self, state_dict):
        self.offset = state_dict["offset"]

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()